# Improve the Microscope Pictures with a Camera, a NeoPixel Illuminator and Fourier Ptychography

by [Jose David Marroquin Toledo](https://github.com/josemarroquintoledo/)

## 1 Import Functions from Other Jupyter Notebooks

In [ ]:
%run ../jupyter-modules/fwdimaging.ipynb
%run ../jupyter-modules/phaseretrieval.ipynb
%run ../jupyter-modules/microscopx.ipynb
%run ../jupyter-modules/fputilities.ipynb

## 2 Takes the Photos

In [ ]:
N_PHOTOS = 47
CAM = 0  # Run ls -ltrh /dev/video* to list the video devices picked up
         # by the Linux kernel.

mode = 'RGB'  # 'RGB', 'GRB', 'R', 'G', 'B' or 'W'.
project_path = take_photos(CAM, N_PHOTOS, color=mode)

## 3 Generate the Incident Wave Vectors for the NeoPixel Lamp

In [ ]:
# SAMPLE_DISTANCE = 47  # Distance between in mm between the NeoPixel
                      # illuminator and the sample.
# {<Pixel shape>: <angle in degrees measured from the x-axis
#                  for the first Pixel>, ... }
PIXEL_LAMP = {7: 0, 16: -11.25, 24: -7.5}

xy_lst_per_ring = get_pixels_xy(rings=PIXEL_LAMP)
xy_lst_lst= sorted(xy_lst_per_ring.values())
xy_lst = list()
for lst in xy_lst_lst:
    # Concatenate all list of xy_lst_lst.
    xy_lst.extend(lst)
wv_xy = gen_wave_vectors(xy_lst, SAMPLE_DISTANCE)

## 4 Import the Lo-Res Image Set

In [ ]:
lores_set = read_lores_img_set(project_path, N_PHOTOS,
                               img_prefix='IMG_' + mode.upper())
hires_w_px = lores_set.shape[2] * 4  # Width of the reconstruction in px.
hires_h_px = lores_set.shape[1] * 4  # Height of the reconstruction in
                                     # px.

## 5 Set a Reconstruction Sequence

In [ ]:
# Generate a sequence that starts in the center of the Fourier spectrum
# (central Pixel).
seq = list(range(1, N_PHOTOS + 1))

## 6 Reconstruct the Hi-Res Image from the Lo-Res Image Set

In [ ]:
# Pixel's datasheet for wavelengths in nanometers:
# http://www.seeedstudio.com/document/pdf/WS2812B%20Datasheet.pdf
PX_COLORS_WAVELEN = {'r': [620, 630], 'g': [515, 530],
                     'b': [465, 475]}
# CCD_PX = 2.75e-6  # The size of the employed CCD in m.
# NA = 0.08  # Numerical aperture (NA) of the objective lens.

mode = mode.lower()
lst_path = list()
if mode == 'gbr':
    mode = 'rgb'
for i in range(len(mode)):
    print('<HiResReconstruction-' + mode[i].upper() + '>')
    color = PX_COLORS_WAVELEN[mode[i]]
    wavelen = (sum(color) / len(color)) * 1e-9  # In nanometers.
    # Generate the coherent transfer function of the coherent imaging
    # system.
    cft, lores_w_px, lores_h_px, dkx, dky, kx, ky = gen_cft(np.array(wv_xy),
        wavelen, CCD_PX, NA, hires_w_px, hires_h_px)
    if mode == 'rgb':
        img_set = lores_set[:, :, :, i]
    else:
        img_set = lores_set
    # Reconstruct the Hi-Res image.
    path = gen_hires_img(img_set, seq, hires_w_px, hires_h_px, cft,
                         lores_w_px, lores_h_px, dkx, dky, kx, ky,
                         project_path, name=('IMG_FP_' +
                                             mode[i].upper()))
    # Add the path of the reconstruction to a list. This follows the
    # same order of mode (string).
    lst_path.append(path)

In [ ]:
# If the photos were taken in RGB mode, merge.
if mode == 'rgb':
    imgs = list(map(lambda path: Image.open(path), lst_path))
    rgb_reconstruction = Image.merge('RGB', tuple(imgs))
    rgb_reconstruction.save(Path(lst_path[0]).parent.as_posix() +
                            '/IMG_FP_RGB.TIF')
    map(lambda pil_img: pil_img.close(), imgs)